In [50]:
__author__

import pandas as pd
import numpy as np
import math

class NetworkAddressing():
    '''
    A python class to Subnet Class C Networks
    
    Definitions:
        CIDR: Classless Inter-Domain Routing
        
    '''

    def __init__(self, full_ip):
        # constructor funtion
        self.ip = full_ip.split('/')[0] if '/' in full_ip else full_ip
        self.cidr =int(full_ip.split('/')[1]) if '/' in full_ip else None
        self.check_fullIP_validity()
        self.IPClass = None
        self.partitionOctet = None
        self.bits = [8, 16, 24, 32]
        self.bitForPartition = None
        self.subnetHosts = None
        self.ipProperties = {
            'A': {'Default Subnet': '255.0.0.0',
                 'Default CIDR': 8,
                 },
            'B': {'Default Subnet': '255.255.0.0',
                 'Default CIDR': 16,},
            
            'C': {'Default Subnet': '255.255.255.0',
                 'Default CIDR': 32,}
        }
        self.subnetNetworkAddress = []
        self.subnetBroadcastAddress = []
        
        #---------Calling Methods-----------------
        self.findIPClass()
        self.defaultBinaryIP, self.defaultOctetList = self.IPToBinary(self.ip)
        self.defaultOctetPosition = self.retrieveOctetPosition(self.cidr)
        self.defaultNetworkAddress, self.defaultBroadcastAddress, self.defaultSubnetMask = self.findNetworkBroadcastandSubnetAddresses(self.ip, self.cidr)
        self.firstHost = self.findHostRange(self.defaultNetworkAddress, self.defaultBroadcastAddress)[0]
        self.lastHost = self.findHostRange(self.defaultNetworkAddress, self.defaultBroadcastAddress)[-1]
        self.numberOfHosts = len(self.findHostRange(self.defaultNetworkAddress, self.defaultBroadcastAddress)) + 2


    def check_fullIP_validity(self):
        if self.cidr is None:
            return f'provide a valid IP. {self.ip} does not have cidr'

    def findIPClass(self):
        IPFirstOctet = int(self.ip.split('.')[0])
        if IPFirstOctet < 128:
            self.IPClass = 'A'
        elif IPFirstOctet >= 128 and IPFirstOctet < 192:
            self.IPClass = 'B'
        elif IPFirstOctet >= 192 and IPFirstOctet < 224:
            self.IPClass = 'C'
        elif IPFirstOctet >= 224 and IPFirstOctet < 240:
            self.IPClass = 'Multicast'
        else:
            self.IPClass = 'Reserved'

    def retrieveOctetPosition(self, cidr):
        
        if cidr < 0:
            print('invalid Network Bits')
            return
            
        for i in range(len(self.bits)):
            if cidr < self.bits[i]:
                return i
    
    def paddedZeroes(self, binaryNumberString):
        biLength = len(binaryNumberString)
        zeroesToPad = (8 - biLength) * '0'
        binaryNumberString = zeroesToPad + binaryNumberString
        return binaryNumberString

    
    def binaryToDecimal(self, binaryNumber):
        ''' method to convert binary number to decimal value
        param:
            binaryNumber (int): the binary number to be converted
        returns:
            decimalNumber (int): the converted number'''

        decimalNumber = 0
        bits = range(7, -1, -1)
        for bit, i in zip(bits, range(0, 8)):
            decimalNumber = decimalNumber + int(binaryNumber[i]) * 2**bit
        return decimalNumber

    
    def convertDecimalToAnotherBase(self, baseNumber, numberToConvert):
        ''' method to convert base 10 numbers (decimal) to other bases
        param:
            baseNumber (int): the base to be considered
            numberToConvert (int): the base 10 number to be converted
        returns:
            baseString (str): the converted base number
            '''
        
        baseString = ''
        while True:
            if numberToConvert < 0: # treat negative numbers
                print('Cannot handle negative numbers')
                break
            
            else:
                if numberToConvert < baseNumber:
                    baseString += str(numberToConvert)
                    break
                else:
                    modNumber = numberToConvert % baseNumber
                    baseString += str(modNumber)
                    numberToConvert //= baseNumber
        baseString = baseString[::-1]
        return baseString
    
    def longIP(self, IP):
        octetList = self.IPToBinary(IP)[1]
        IPMerged=('').join(octetList)
        return IPMerged

    def insertDotInIP(self, longIP):
        return '.'.join(longIP[i:i+8] for i in range(0, len(longIP), 8))

    def breakIPsIntoOctets(self, longIPs):
        dottedIPs = self.insertDotInIP(longIPs)
        return [x for x in dottedIPs.split('.')]

    def findNetworkBroadcastandSubnetAddresses(self, IP, cidr):
        longIP = self.longIP(IP)
        networkAddress = longIP[:cidr] + '0' * (len(longIP)-cidr)
        networkAddress = [str(self.binaryToDecimal(x)) for x in self.breakIPsIntoOctets(networkAddress)]
        networkAddress = '.'.join(networkAddress)
        
        broadcastAddress = longIP[:cidr] + '1' * (len(longIP)-cidr)
        broadcastAddress = [str(self.binaryToDecimal(x)) for x in self.breakIPsIntoOctets(broadcastAddress)]
        broadcastAddress = '.'.join(broadcastAddress)

        subnetMaskAddress = cidr * '1' + '0'* ((8*4)-cidr)
        subnetMaskAddress = [str(self.binaryToDecimal(x)) for x in self.breakIPsIntoOctets(subnetMaskAddress)]
        subnetMaskAddress = '.'.join(subnetMaskAddress)

        return networkAddress, broadcastAddress, subnetMaskAddress


    
    def IPToBinary(self, IP):
        listOfIPs = [int(x) for x in IP.split('.')]
        octetList = [self.paddedZeroes(self.convertDecimalToAnotherBase(2, x)) for x in listOfIPs]
        binaryIP = '.'.join(octetList)
        return binaryIP, octetList


    def findHostRange(self, networkAddress, broadcastAddress):
        hostsLastIPs = [str(x) for x in range(int(networkAddress.split('.')[-1]) +1, int(broadcastAddress.split('.')[-1]))]
        hosts = [networkAddress.replace(networkAddress.split('.')[-1], x) for x in hostsLastIPs]
        return hosts
        
    
    def replaceLastOctet(self, IP, newOctet):
        lastOctet = IP.split('.')[-1]
        newIP = IP[:-len(lastOctet)] + str(newOctet)
        return newIP
    
    def updateAddress(self, IP):
        last = IP.split('.')[-1]
        integerLast = int(last)
        addition = integerLast + 1
        newIP = IP[:-len(last)] + str(addition)
        return newIP


    def subNet(self, numberOfSubnets):
        numberOfBits = math.log(numberOfSubnets, 2)
        subnetCIDR = int(self.cidr + numberOfBits)
        self.subnetHosts = int(self.numberOfHosts / numberOfSubnets)

        octetPosition = self.retrieveOctetPosition(subnetCIDR)

        defaultNetworkAddressLastOctet = self.defaultNetworkAddress.split('.')[octetPosition]
        defaultBroadcastAddressLastOctet = self.defaultBroadcastAddress.split('.')[-1]
        
        subNetDictionary = {}
            

        networkAddresses = [self.defaultNetworkAddress[:-len(defaultNetworkAddressLastOctet)]+str(x) for x in range(int(defaultNetworkAddressLastOctet), int(defaultBroadcastAddressLastOctet), self.subnetHosts)]
        broadcastAddresses = [self.findNetworkBroadcastandSubnetAddresses(x, subnetCIDR)[1] for x in networkAddresses]
        subnetMasks = [self.findNetworkBroadcastandSubnetAddresses(x, subnetCIDR)[2] for x in networkAddresses]
        subnetData = pd.DataFrame(columns=['SubnetID', "Network Address", 'FirstHost Address', 'LastHost Address', 'Broadcast Address', 'SubnetMask Address', 'CIDR'])
        
        for i in range(numberOfSubnets):
            subnetID = f'SN{i+1}'
            nAddress = networkAddresses[i]
            fHost = self.findHostRange(networkAddresses[i], broadcastAddresses[i])[0]
            lHost = self.findHostRange(networkAddresses[i], broadcastAddresses[i])[-1]
            bAddress = broadcastAddresses[i]
            subnetMask = subnetMasks[i]
            CIDR = nAddress + '/' + str(int(subnetCIDR))

            subnetData.loc[i] = [subnetID, nAddress, fHost, lHost, bAddress, subnetMask, CIDR]#, subnetMask]
        
        
        subnetData.set_index('SubnetID', inplace=True)
        print(f"Number of Hosts in each subnet = {self.subnetHosts-2}")
        return subnetData

In [45]:
ip = '192.124.8.95/26'
#cidr = 26

In [46]:
networkAddress = NetworkAddressing(ip)

In [47]:
networkAddress.defaultOctetPosition

3

In [48]:
networkAddress.defaultBroadcastAddress

'192.124.8.127'

In [49]:
networkAddress.subNet(4)

Number of Hosts in each subnet = 14


,Network Address,FirstHost Address,LastHost Address,Broadcast Address,SubnetMask Address,CIDR
SubnetID,,,,,,
SN1,192.124.8.64,192.124.8.65,192.124.8.78,192.124.8.79,255.255.255.240,192.124.8.64/28
SN2,192.124.8.80,192.124.8.81,192.124.8.94,192.124.8.95,255.255.255.240,192.124.8.80/28
SN3,192.124.8.96,192.124.8.97,192.124.8.110,192.124.8.111,255.255.255.240,192.124.8.96/28
SN4,192.124.8.112,192.124.8.113,192.124.8.126,192.124.8.127,255.255.255.240,192.124.8.112/28


In [371]:
for i in range(3, 4):
    print(i)

3


In [3]:
x = '192.34/5'

In [6]:
y = '/' in x

In [7]:
y

True